In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.common.exceptions import NoSuchElementException   
import progressbar

options = Options()
options.headless = True
options.add_argument("--window-size=1920,20200") #this is important, to tell it how much of the webpage to import
driver = webdriver.Chrome(options=options, executable_path=r'/usr/local/bin/chromedriver')

In [68]:
def formazioni(competizione = 'campionato'):
    #if competizione == 'campionato':
    link = 'https://leghe.fantacalcio.it/fantapalla-forever/formazioni?id=185855'
    driver.get(link)

    players = {}
    for l in [2,3,4,5]:
        for k in [1,2]:
            name = driver.find_element_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[1]/div["+str(k)+"]/div/div[2]/h4").text


            all_playersE = driver.find_elements_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[2]/div["+str(k)+"]/table[1]/tbody/tr[@class='player-list-item even  ']/td[1]/span/span[2]/a")
            all_playersO = driver.find_elements_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[2]/div["+str(k)+"]/table[1]/tbody/tr[@class='player-list-item odd  ']/td[1]/span/span[2]/a")
            all_playersEP = driver.find_elements_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[2]/div["+str(k)+"]/table[2]/tbody/tr[@class='player-list-item even  ']/td[1]/span/span[2]/a")
            all_playersOP = driver.find_elements_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[2]/div["+str(k)+"]/table[2]/tbody/tr[@class='player-list-item odd  ']/td[1]/span/span[2]/a")
            #all_playersE = driver.find_elements_by_xpath("")

            all_pl = all_playersE + all_playersO + all_playersEP + all_playersOP
            names = []
            for pl in all_pl:
                names.append((pl.text).upper())
            players[name] = names
    return pd.DataFrame.from_dict(data=players, orient='index').T

In [70]:
formazioni()

,AS 800A,PDG 1908,Ignoranza Everywhere,soros fc,Mainz na gioia,Palla Pazza,I Diseredati,XYZ
0,HANDANOVIC,None,SZCZESNY,OSPINA,None,MIRANTE,None,SILVESTRI
1,SMALLING,None,HATEBOER,BONUCCI,None,GOSENS,None,BRUNO ALVES
2,CUADRADO,None,BIRAGHI,PASALIC,None,CHIELLINI,None,DJURICIC
3,RAMIREZ,None,VERETOUT,KUCKA,None,RUIZ,None,LOCATELLI
4,SIMEONE,None,BARROW,LAPADULA,None,ORSOLINI,None,BELOTTI
5,RONALDO,None,CAPUTO,LUKAKU,None,LASAGNA,None,RIBERY
6,DANILO,None,KOULIBALY,FERRARI G.,None,PEZZELLA GER.,None,DI LORENZO
7,COLLEY,None,TOMIYASU,MULDUR,None,GODIN,None,ROMAGNOLI
8,KULUSEVSKI,None,PULGAR,ARTHUR,None,MALINOVSKYI,None,CHIESA
9,PELLEGRINI LO.,None,MKHITARYAN,CASTROVILLI,None,LAZOVIC,None,TONALI
